In [9]:
import os
from collections import defaultdict

catalog = defaultdict(dict)
with open('catalog.txt', 'r') as file:
    k = 0
    for line in file:
        
        line = line.replace('\n', '').split(';')
        title = line[3].lower().strip()
        fid = line[8]
        author = line[0].lower().strip()
        if title in catalog and author in catalog[title]:
            k += 1
        catalog[title].update({author: fid})

In [10]:
dir_in = 'plain_texts_cut'
fid_to_title = {}
found= 0
not_found = 0
tie = 0
no_author = 0
for filename in os.listdir(dir_in):
    title = filename.rstrip('plain')
    if title in catalog:
        possible_authors = []
        with open(dir_in + '/' + filename) as file:
            content = file.read(500).lower()
            for author in catalog[title]:
                if author != '' and author in content:
                    possible_authors.append(author)
        if not possible_authors:
            no_author += 1
            continue
        elif len(possible_authors) > 1:
            tie += 1
        else:
            found += 1
        fid = catalog[title][possible_authors[0]]
        fid_to_title[fid] = title
    else:
        not_found += 1

In [11]:
no_author, not_found, tie, len(fid_to_title)

(292, 221, 1002, 37905)

In [1]:
import requests
from lxml import html
from multiprocessing import Pool, Manager
from multiprocessing.pool import ThreadPool
import time

In [2]:
requests.get(f'http://proxy.flibusta.site').status_code

204

In [13]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.75 Safari/537.36'}

In [14]:
mng = Manager()
fid_to_genres = mng.dict()

In [24]:
def process_fid(tup):
    i, fid = tup
    if i % 500 == 0:
        print(i, len(fid_to_genres))
    if fid in fid_to_genres and len(list(fid_to_genres[fid])) > 0:
        return
    fid_to_genres[fid] = mng.list()
    e = requests.get(f'http://proxy.flibusta.site/b/{fid}', 
                    headers=headers)
    assert e.status_code == 200, e
    e = html.fromstring(e.text)
    e = e.get_element_by_id('main').find_class('genre')
    for g in e:
        if g.get('name'):
            fid_to_genres[fid].append(g.get('name'))
    time.sleep(2)

In [28]:
%%time
p = Pool(7)
p.map(process_fid, sorted(list(enumerate(fid_to_title.keys()))))
p.terminate()
p.join()

0 37771
5500 37771
3000 37771
4500 37771
6000 37771
500 37771
3500 37771
4000 37771
9500 37771
1500 37771
8500 37771
9000 37771
7000 37771
7500 37771
8000 37771
12500 37771
13000 37771
11000 37771
13500 37771
11500 37771
5000 37771
12000 37771
15000 37771
16500 37771
6500 37771
17000 37771
17500 37771
18000 37771
14000 37771
1000 37771
10000 37771
19000 37771
10500 37771
18500 37771
2000 37771
2500 37771
15500 37771
14500 37771
24500 37771
19500 37771
20500 37771
21000 37771
21500 37771
27500 37771
22000 37771
23500 37771
25000 37771
26000 37771
20000 37771
28000 37771
28500 37771
30000 37771
30500 37771
31000 37771
31500 37771
16000 37771
22500 37771
32500 37771
23000 37771
25500 37771
26500 37771
35500 37771
27000 37771
29000 37771
37000 37771
29500 37771
37500 37771
32000 37771
24000 37771
34000 37771
33000 37771
33500 37771
36000 37771
34500 37773
36500 37774
35000 37775
CPU times: user 2.99 s, sys: 1.82 s, total: 4.81 s
Wall time: 6min 4s


In [29]:
import json

In [30]:
json.dump({k:list(v) for k, v in fid_to_genres.items()}, open( "all genres.json", 'w' ))

In [23]:
mng = Manager()
fid_to_genres = mng.dict(json.load(open("37175 genres.json")))

In [32]:
fid_to_genres = json.load(open("all genres.json"))

In [33]:
fid_to_genres

{'29350': ['child_adv'],
 '445558': ['nonf_biography'],
 '138818': ['sci_psychology'],
 '106311': ['detective'],
 '276628': ['child_tale'],
 '110223': ['det_irony'],
 '258466': ['russian_fantasy', 'sf_humor'],
 '158785': ['love_contemporary'],
 '483606': ['architecture_book'],
 '50791': ['sf_fantasy'],
 '218380': ['nonfiction'],
 '231906': ['prose_military'],
 '324264': ['prose_contemporary'],
 '26925': ['love_history'],
 '39343': ['sf', 'sf_fantasy'],
 '221664': ['sf_action'],
 '394483': ['det_classic'],
 '270095': ['love_short'],
 '302344': ['love_sf'],
 '79847': ['sf_humor'],
 '348811': ['love_contemporary'],
 '326495': ['detective'],
 '296828': ['sf_heroic', 'sf_fantasy'],
 '323262': ['prose_contemporary'],
 '57748': ['detective'],
 '452331': ['sf_fantasy'],
 '256158': ['sf_fantasy'],
 '246298': ['det_police'],
 '167922': ['love_short'],
 '340052': ['love_history', 'love_short'],
 '97570': ['sf_humor'],
 '289212': ['sci_history', 'sci_philosophy', 'religion_esoterics'],
 '328531': 

In [ ]:
def process_link():
    for link in links:
        response = requests.get(link)
        assert response.status_code == 200
        dom = html.fromstring(response.text)
        for href in dom.xpath('//a/@href'):
            if 'wiki' not in href:
                continue
            norm_href = unquote(href).split('#')[0]
            all_hrefs[link].add(norm_href)
            if norm_href in hrefs_normalized and norm_href not in link:
                res[link].add(norm_href)
        time.sleep(1)
    return res, all_hrefs

In [21]:
from multiprocessing import Pool, Manager
d = Manager().dict()
def f(x):
    global d
    d[x] = x

p = Pool(6)

p.map(f, list(range(10000)))
p.terminate()
p.join()

In [25]:
len(d.keys())

10000

In [68]:
def it():
    for filename in os.listdir('plain_texts_cut'):
        with open('plain_texts_cut/' + filename) as file:
            for line in file:
                yield filename + line

In [69]:
x = it()

In [89]:
next(x)

'про меняplain Про меня (fb2) - Про меня 998K скачать: (fb2) - (epub) - (mobi) - Елена Колина Елена Колина Про меня Я посвящаю эту книгу моей любимой подруге Аньке. Все совпадения случайны, все персонажи, и знаменитая писательница, и главный режиссер, лишь повод для того, чтобы рассказать о любви и ревности одной странной девочки, такой же странной, как вы и я. Люди делятся на скучных и странных. Скучные живут свои скучные жизни вдали от театра, а странные имеют все шансы на то, чтобы стать персонажами пьесы. Мир существует для того, чтобы войти в книгу, мир существует, чтобы войти в пьесу. Я все записываю, а потом напишу пьесу «Из жизни странных людей». Они и не знают, что я все записываю! Все знают, что все уже было, все умное уже однажды кто-то подумал, все глупое уже кто-то высказал, любое чувство кто-то испытал, любую ошибку кто-то совершил. И несчетное количество раз кто-то сказал себе «я думал, что все понимаю, а я ничего не понимаю» и попросил прощения. Но ведь для каждого его 